In [113]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, LSTM, Bidirectional
from keras.models import Sequential, Model
from keras.optimizers import Adam

import nltk
import spacy
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

nlp= spacy.load('en_core_web_sm')

In [54]:
file_path= r"D:\Datasets\French-eng Translator\fra-eng\fra.txt"


In [55]:
# static data
batch_size= 64  
epochs= 10
latent_dim= 100     # Latent dimensionality of encoding space
num_samples= 10000  # num of samples to train on


In [56]:
# Vectorize the data
input_texts= []
target_texts= []

input_charset= set()
target_charset= set()


with open(file_path, "r", encoding= 'utf-8') as f:
    lines= f.read().split('\n')
    

In [57]:
print('length of lines in the file:',len(lines))
lines

length of lines in the file: 167131


['Go.\tVa !',
 'Hi.\tSalut !',
 'Run!\tCours\u202f!',
 'Run!\tCourez\u202f!',
 'Who?\tQui ?',
 'Wow!\tÇa alors\u202f!',
 'Fire!\tAu feu !',
 "Help!\tÀ l'aide\u202f!",
 'Jump.\tSaute.',
 'Stop!\tÇa suffit\u202f!',
 'Stop!\tStop\u202f!',
 'Stop!\tArrête-toi !',
 'Wait!\tAttends !',
 'Wait!\tAttendez !',
 'Go on.\tPoursuis.',
 'Go on.\tContinuez.',
 'Go on.\tPoursuivez.',
 'Hello!\tBonjour !',
 'Hello!\tSalut !',
 'I see.\tJe comprends.',
 "I try.\tJ'essaye.",
 "I won!\tJ'ai gagné !",
 "I won!\tJe l'ai emporté !",
 'I won.\tJ’ai gagné.',
 'Oh no!\tOh non !',
 'Attack!\tAttaque !',
 'Attack!\tAttaquez !',
 'Cheers!\tSanté !',
 'Cheers!\tÀ votre santé !',
 'Cheers!\tMerci !',
 'Cheers!\tTchin-tchin !',
 'Get up.\tLève-toi.',
 'Go now.\tVa, maintenant.',
 'Go now.\tAllez-y maintenant.',
 'Go now.\tVas-y maintenant.',
 "Got it!\tJ'ai pigé !",
 'Got it!\tCompris !',
 'Got it?\tPigé\u202f?',
 'Got it?\tCompris\u202f?',
 "Got it?\tT'as capté\u202f?",
 'Hop in.\tMonte.',
 'Hop in.\tMontez.',
 'Hu

In [58]:
for line in lines[: min(num_samples, len(lines)-1)]:

    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" char for the targets, and "\n" as "end sequence" char.
    target_text= '\t' + target_text + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_charset:
            input_charset.add(char)
            
    for char in target_text:
        if char not in target_charset:
            target_charset.add(char)
        

In [59]:
target_texts[:5]

['\tVa !\n',
 '\tSalut !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n']

In [60]:
input_charset

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [61]:
len(input_charset), len(target_charset)

(70, 93)

In [62]:
len(input_texts), len(target_texts)

(10000, 10000)

In [63]:
# create a sorted list of set of unique words or phrases
input_charset= sorted(list(input_charset))
target_charset= sorted(list(target_charset))

num_encoder_tokens= len(input_charset)
num_decoder_tokens= len(target_charset)

max_encoder_seq_length= max([len(txt) for txt in input_texts])
max_decoder_seq_length= max([len(txt) for txt in target_texts])

In [64]:
num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length

(70, 93, 16, 59)

In [65]:
# sort the input charset 
input_charset[:5]

[' ', '!', '$', '%', '&']

In [66]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [75]:
# create a dictionary of words with their indexes

input_char_token_dict= dict([(word, i) for i, word in enumerate(input_charset)])
target_char_token_dict= dict([(word, i) for i, word in enumerate(target_charset)])

In [76]:
len(input_char_token_dict), len(target_char_token_dict)

(70, 93)

In [77]:

encoder_input_data= np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32')
decoder_input_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')
decoder_target_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')




In [78]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((10000, 16, 70), (10000, 59, 93), (10000, 59, 93))

In [127]:
for i, (input_text, target_text) in enumerate(zip(input_texts[:3], target_texts[3])):
#     print(input_text)
    for t, char in enumerate(input_text):
        print(i, t, char, input_char_token_dict[char] )
        encoder_input_data[i, t, input_char_token_dict[char]]= 1.
#     print(i, t + 1, char, input_char_token_dict[' '] )
    encoder_input_data[i, t + 1, input_char_token_dict[' ']]= 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep.
        decoder_input_data[i, t, target_char_token_dict[char]]= 1.
        
#         print(t, char)

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include start seq
            decoder_target_data[i, t - 1, target_char_token_dict[char]]= 1.
            
    decoder_input_data[i, t + 1, target_char_token_dict[' ']]= 1.
    decoder_target_data[i, t:, target_char_token_dict[' ']]= 1.

0 0 G 26
0 1 o 58
0 2 . 8
0 3 . 0
1 0 H 27
1 1 i 52
1 2 . 8
1 3 . 0
2 0 R 37
2 1 u 64
2 2 n 57
2 3 ! 1
2 4 ! 0


In [109]:
# Define and input seq and process it

encoder_inputs= Input(shape= (None, num_encoder_tokens))
encoder= LSTM(latent_dim, return_state= True)
encoder_outputs, state_h, state_c= encoder(encoder_inputs)

# we discard 'encoder_outputs' and we will use only state
encoder_states= [state_h, state_c]

In [110]:
encoder_outputs, encoder_states

(<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm_3')>,
 [<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm_3')>,
  <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm_3')>])

In [111]:
# create decoder using encoders states
decoder_inputs= Input(shape= (None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder= LSTM(latent_dim, return_state= True, return_sequences= True)
decoder_outputs, _, _= decoder(decoder_inputs)

decoder_dense= Dense(num_decoder_tokens, activation= 'softmax')
decoder_outputs= decoder_dense(decoder_outputs)

In [114]:
#Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [116]:
model.compile(optimizer= 'rmsprop', loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs= epochs, validation_split= 0.2)



Epoch 1/10


125/125 [==============================] - 6s 33ms/step - loss: 1.2317 - accuracy: 0.0425 - val_loss: 1.0177 - val_accuracy: 0.0378
Epoch 2/10
125/125 [==============================] - 4s 29ms/step - loss: 1.1451 - accuracy: 0.0448 - val_loss: 1.0097 - val_accuracy: 0.0381
Epoch 3/10
125/125 [==============================] - 4s 29ms/step - loss: 1.1234 - accuracy: 0.0452 - val_loss: 0.9998 - val_accuracy: 0.0383
Epoch 4/10
125/125 [==============================] - 4s 29ms/step - loss: 1.1088 - accuracy: 0.0453 - val_loss: 0.9707 - val_accuracy: 0.0384
Epoch 5/10
125/125 [==============================] - 4s 28ms/step - loss: 1.0955 - accuracy: 0.0457 - val_loss: 0.9595 - val_accuracy: 0.0393
Epoch 6/10
125/125 [==============================] - 4s 29ms/step - loss: 1.0863 - accuracy: 0.0465 - val_loss: 0.9635 - val_accuracy: 0.0405
Epoch 7/10
125/125 [==============================] - 4s 29ms/step - loss: 1.0751 - accuracy: 0.0473 - val_loss: 0.9422 - val_accuracy: 0.0

In [ ]:
# Next: inference mode (sampling).

# Here's the drill:

# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states


In [128]:
# Define sampling models
encoder_model= Model(encoder_inputs, encoder_states)

In [129]:
decoder_state_input_h= Input(shape= (latent_dim,))
decoder_state_input_c= Input(shape= (latent_dim,))
decoder_state_inputs= [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c= decoder(decoder_inputs, initial_state= decoder_state_inputs)

decoder_states= [state_h, state_c]
decoder_outputs= decoder_dense(decoder_outputs)
decoder_model= Model([decoder_inputs]+decoder_state_inputs, [decoder_outputs]+decoder_states)


In [131]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_char_token_dict.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_char_token_dict.items())


In [135]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_char_token_dict['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [136]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Hi.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 18ms/step
-
Input sentence: Run!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Who?
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  


1/1 [==============================] - 0s 14ms/step
-
Input sentence: Wow!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Fire!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 14ms/step
-
Input sentence: Help!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 15ms/step
-
Input sentence: Jump.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 31ms/step
-
Input sentence: Stop!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  


1/1 [==============================] - 0s 22ms/step
-
Input sentence: Stop!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 14ms/step
-
Input sentence: Stop!
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Be nice.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Be nice.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Be nice.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  


1/1 [==============================] - 0s 21ms/step
-
Input sentence: Be nice.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 17ms/step
-
Input sentence: Be nice.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step
-
Input sentence: Beat it.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 31ms/step
-
Input sentence: Call me.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 17ms/step
-
Input sentence: Call me.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  


1/1 [==============================] - 0s 20ms/step
-
Input sentence: Call us.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 21ms/step
-
Input sentence: Call us.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 28ms/step
-
Input sentence: Come in.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Come in.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Come in.
Decoded sentence: e  s s e  e e is  i      e  e e ee  e e  e  e  e  e  e   e  
